In [3]:
import pandas as pd
from google.colab import files
uploaded = files.upload()
df = pd.read_csv("dataset_clean_normalized.csv")
df.head()
print(df.dtypes)

Saving dataset_clean_normalized.csv to dataset_clean_normalized.csv
timestamp               object
WeekDay                float64
CPU_Usage_Pct          float64
RAM_Usage_Pct          float64
ESP32_Temperature_C    float64
humidity               float64
Active_Power_W         float64
Active_Energy_kWh      float64
Voltage_V              float64
Current_A              float64
Frequency_Hz           float64
Power_Factor           float64
CPU_Power_Pct          float64
CPU_Temp_Pct           float64
energy_source           object
dtype: object


In [4]:
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

# Supprimer les colonnes non numériques
df_features = df.drop(columns=["timestamp", "energy_source"])
# Initialiser le modèle
iso_forest = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)

# Entraîner et prédire
df["anomaly"] = iso_forest.fit_predict(df_features)

# Statistiques
print(df["anomaly"].value_counts())
anomalies = df[df["anomaly"] == -1]
print("Number of anomalies detected :", len(anomalies))
anomalies.head()
# Exporter vers un nouveau fichier CSV
df.to_csv("dataset_with_anomalies.csv", index=False)

# Télécharger dans ton PC
files.download("dataset_with_anomalies.csv")


anomaly
 1    175204
-1      9222
Name: count, dtype: int64
Number of anomalies detected : 9222


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
# Initialize the anomaly_type column
df["anomaly_type"] = "normal"

# Apply rules only to detected anomalies
anomalies = df[df["anomaly"] == -1]

# Define business rules for labeling anomaly types
for index, row in anomalies.iterrows():
    if row["Active_Power_W"] > 0.75 and row["CPU_Usage_Pct"] < 0.1:
        df.at[index, "anomaly_type"] = "overconsumption"
    elif row["Voltage_V"] > 0.78:
        df.at[index, "anomaly_type"] = "overvoltage"
    elif row["Voltage_V"] < 0.74:
        df.at[index, "anomaly_type"] = "undervoltage"
    elif row["Current_A"] > 0.80:
        df.at[index, "anomaly_type"] = "high current"
    elif row["ESP32_Temperature_C"] > 0.7:
        df.at[index, "anomaly_type"] = "ESP32 overheating"
    elif row["Frequency_Hz"] < 0.65:
        df.at[index, "anomaly_type"] = "unstable frequency"
    elif row["energy_source"] == "battery":
        df.at[index, "anomaly_type"] = "suspicious battery switch"
    else:
        df.at[index, "anomaly_type"] = "unknown"
# Save to new CSV
df.to_csv("dataset_with_anomaly_types.csv", index=False)

# Download it from Colab
from google.colab import files
files.download("dataset_with_anomaly_types.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>